# C02 prediction
This notebook introduces the predicition of CO2 concentration, using the available data on [Kaggle](https://www.kaggle.com/datasets/ulrikthygepedersen/co2-emissions-by-country).
It will provide some prediction models for France and China.

All models will be evaluated using the `score` method from [`scikit-learn`](https://scikit-learn.org/stable/), which will also provide the models.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model as lin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from IPython.display import display
import numpy as np
%matplotlib inline

## Data visualisation

### Data preparation

In [ ]:
# Size of the file

file_stat = os.stat("data/co2_emissions_kt_by_country.csv")
print(f"File size: {file_stat.st_size / 1024} kB.")

df = pd.read_csv("data/co2_emissions_kt_by_country.csv")
display(df.head(10))

# Removing NA
print(f"Number of NA: {df.isna().sum().sum()}.")
if df.isna().sum().sum() > 0:
    print(f"Before removing NA: {df.shape}.")
    df.dropna(axis = 0)
    print(f"After removing NA: {df.shape}.")
else:
    print(f"Data frame shape: {df.shape}.")

print(f"Number of countries: {df['country_code'].nunique()}.")

The current dataframe holds data for 255 countries, each line is for a specific country during a specific year.
We will first change this into a dictonnary, indexed by the country codes.

In [ ]:
df_dict = {}
for k in df["country_code"]:
    df_dict[k] = df[df["country_code"] == k].copy()
    df_dict[k].drop("country_code", axis = 1, inplace = True)

print(f"We have a set of {len(df_dict)} countries.")
print("The keys are the country codes.\nNow, we have separated the data for each country.")

print(f"The countries are:")
for k in df_dict:
    print(f"\t{k} - {df_dict[k]['country_name'].iloc[0]}")

### Data for a few countries, as example
We are going to make a first plot of the evolution using the data for France and a few other countries.

In [ ]:
plt.figure(figsize = (15, 9))
plt.plot(df_dict["FRA"]["year"],df_dict["FRA"]["value"], label = "France")
plt.plot(df_dict["USA"]["year"],df_dict["USA"]["value"], label = "USA")
plt.plot(df_dict["CHN"]["year"],df_dict["CHN"]["value"], label = "China")
plt.plot(df_dict["IND"]["year"],df_dict["IND"]["value"], label = "India")
plt.plot(df_dict["RUS"]["year"],df_dict["RUS"]["value"], label = "Russia")
plt.legend()
plt.show()

Obviously, the data is not linear, so we will use a polynomial regression to fit the data.
However, unfortunately, for China, an exponential regression is more appropriate.

## Training for France
### Linear regression
Let's first try to train a linear model for France, even if the data are absolutely not linear.
But first, we need a few information about the data.

In [ ]:
print("10 first lines of the data frame for France:")
display(df_dict["FRA"].head(10))
print("10 last lines of the data frame for France:")
display(df_dict["FRA"].tail(10))
print("Description of the data frame for France:")
df_dict["FRA"].describe()

Now, we can prepare the data for the training.

In [ ]:
models = {}
models["FRA"] = lin.LinearRegression()
X_train, X_test, Y_train, Y_test = train_test_split(df_dict["FRA"]["year"], df_dict["FRA"]["value"], test_size = 0.2)
X_train = X_train.values.reshape(-1, 1)
Y_train = Y_train.values.reshape(-1, 1)
X_test = X_test.values.reshape(-1, 1)
Y_test = Y_test.values.reshape(-1, 1)

models["FRA"].fit(X_train, Y_train)
prediction = models["FRA"].predict(X_test)
print(f"Score for France: {models['FRA'].score(X_test, Y_test)}")

plt.figure(figsize = (15, 9))
plt.plot(X_test, prediction, label = "Prediction")
plt.scatter(X_test, Y_test, label = "Real", color = "red")
plt.legend()
plt.show()

Regarding the data for France, we can see that the data is not linear, so we will use a polynomial regression. The scores confirms that.
### Polynomial regression

In [ ]:
degrees = range(2,10)
scores = []

models_FRA = []

for d in degrees:
    poly = PolynomialFeatures(degree = d, include_bias = False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.fit_transform(X_test)
    models_FRA.append(lin.LinearRegression())
    models_FRA[d - 2].fit(X_train_poly, Y_train)
    scores.append(models_FRA[d - 2].score(X_test_poly, Y_test))

for d in degrees:
    print(f"Score for degree {d}: {scores[d - 2]}")

print(f"Chosen degree: {2 + scores.index(max(scores))}")
models["FRA"] = models_FRA[scores.index(max(scores))]

poly = PolynomialFeatures(degree = 2 + scores.index(max(scores)), include_bias = False)
X_test_poly = poly.fit_transform(X_test)
prediction = models["FRA"].predict(X_test_poly)

prediction_list = []
for i in range(len(X_test)):
    prediction_list.append((X_test[i][0], prediction[i][0]))

prediction_list.sort(key = lambda x: x[0])

plt.figure(figsize = (15, 9))
plt.scatter(X_test,Y_test, label = "Real")
plt.plot([prediction_list[i][0] for i in range(len(prediction_list))], [prediction_list[i][1] for i in range(len(prediction_list))], label = "Prediction", color = "red")
plt.legend()
plt.show()

Let's make a cross-validation of this model.

In [ ]:
scores_cv = cross_val_score(models["FRA"], X_test_poly, Y_test, cv = 5)
print(f"Scores from cross validation: {scores_cv}.")
print(f"Mean score: {scores_cv.mean()}.")
print(f"Scores standard deviation: {scores_cv.std()}.")

We here notice that the best score is obtained with a degree of 3, but the score is still not very good.

## Training for China
As shown before, China has an exponential growth, so we will use an exponential regression.
First, we need to prepare the data and visualize them.

In [ ]:
print("10 first lines of the data frame for China:")
display(df_dict["CHN"].head(10))
print("10 last lines of the data frame for China:")
display(df_dict["CHN"].tail(10))
print("Description of the data frame for China:")
df_dict["CHN"].describe()

Now, we are ready to train the model.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df_dict["CHN"]["year"], df_dict["CHN"]["value"], test_size = 0.2)
X_train = X_train.values.reshape(-1, 1)
Y_train = np.log(Y_train.values).reshape(-1, 1)
X_test = X_test.values.reshape(-1, 1)
Y_test = np.log(Y_test.values).reshape(-1, 1)


models["CHN"] = lin.LinearRegression()

models["CHN"].fit(X_train, Y_train)
prediction = models["CHN"].predict(X_test)
print(f"Score for China: {models['CHN'].score(X_test, Y_test)}")

prediction_list = []
for i in range(len(X_test)):
    prediction_list.append((X_test[i][0], prediction[i][0]))

prediction_list.sort(key = lambda x: x[0])

plt.figure(figsize = (15, 9))
plt.plot(np.array([prediction_list[i][0] for i in range(len(prediction_list))]).reshape(-1,1), np.exp(np.array([prediction_list[i][1] for i in range(len(prediction_list))])).reshape(-1,1), label = "Prediction")
plt.scatter(X_test, np.exp(Y_test), label = "Real", color = "red")
plt.legend()
plt.show()

This model is much better than the one for France.
We'll now cross validate it.

In [ ]:
scores_cv = cross_val_score(models["CHN"], X_test, Y_test, cv = 5)
print(f"Scores from cross validation: {scores_cv}.")
print(f"Mean score: {scores_cv.mean()}.")
print(f"Scores standard deviation: {scores_cv.std()}.")

Below are the values computed for the model.

In [ ]:
a = models["CHN"].coef_[0][0]
b = models["CHN"].intercept_[0]
print(f'The generated model is: value = {np.exp(b)} * ({np.exp(a)})^year.')